# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 2029, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 2029 (delta 248), reused 323 (delta 188), pack-reused 1618 (from 1)
Receiving objects: 100% (2029/2029), 141.68 MiB | 22.01 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 25.6 MB/s eta 0:00:00


In [ ]:
% cd /kaggle/working/RECsys_Challenge2024
! python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function '__pyx_pf_32MatrixFactorization

In [ ]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/SingleRecommenders'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [ ]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [8]:
config = {
    'model': 'P3alpha',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_P3alpha.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [ ]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [10]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [11]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [13]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [14]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

def objective_function_P3alphaRecommender(optuna_trial):
    
    recommender_instance = P3alphaRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 2, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),

    }

    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [15]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_P3alphaRecommender, n_trials=200)

[I 2024-11-19 00:46:18,521] Using an existing study with name 'hyperparameters_tuning_P3alpha' instead of creating a new one.


P3alphaRecommender: Similarity column 38121 (100.0%), 1956.35 column/sec. Elapsed time 19.49 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 29.34 sec. Users per second: 1213


[I 2024-11-19 00:47:11,474] Trial 200 finished with value: 0.04590572066256699 and parameters: {'topK': 122, 'alpha': 0.31548195470754337, 'normalize_similarity': True, 'implicit': False}. Best is trial 128 with value: 0.05332611477891539.


P3alphaRecommender: Similarity column 38121 (100.0%), 2459.00 column/sec. Elapsed time 15.50 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 29.22 sec. Users per second: 1218


[I 2024-11-19 00:47:57,604] Trial 201 finished with value: 0.05223213867651081 and parameters: {'topK': 26, 'alpha': 0.47360323588904746, 'normalize_similarity': True, 'implicit': True}. Best is trial 128 with value: 0.05332611477891539.


...

[I 2024-11-19 03:38:10,238] Trial 397 finished with value: 0.04874719897791813 and parameters: {'topK': 68, 'alpha': 0.26284766790015845, 'normalize_similarity': True, 'implicit': True}. Best is trial 128 with value: 0.05332611477891539.


P3alphaRecommender: Similarity column 38121 (100.0%), 3611.56 column/sec. Elapsed time 10.56 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 23.40 sec. Users per second: 1521


[I 2024-11-19 03:38:44,478] Trial 398 finished with value: 0.00028209999175022335 and parameters: {'topK': 0, 'alpha': 0.3640756160596335, 'normalize_similarity': True, 'implicit': True}. Best is trial 128 with value: 0.05332611477891539.


P3alphaRecommender: Similarity column 38121 (100.0%), 2018.35 column/sec. Elapsed time 18.89 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 29.53 sec. Users per second: 1205


[I 2024-11-19 03:39:36,974] Trial 399 finished with value: 0.04558183889747153 and parameters: {'topK': 111, 'alpha': 0.9517992592054156, 'normalize_similarity': True, 'implicit': True}. Best is trial 128 with value: 0.05332611477891539.


## Some optuna visualizations on recommender parameters

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [17]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [ ]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = P3alphaRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

P3alphaRecommender: Similarity column 38121 (100.0%), 2192.24 column/sec. Elapsed time 17.39 sec


# Testing

Create the recommendations for the submission. 

In [ ]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

Submission file saved as /kaggle/working/submission_P3alpha.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [ ]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/P3alphaRecommender/best_params_P3alpha.json' updated successfully.


Save the history of the tuned model.

In [ ]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/P3alphaRecommender/history_P3alpha.db' updated successfully.


Save the best trained model and its submission.

In [ ]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/P3alphaRecommender/Submission/submission_P3alpha.csv' updated successfully.
P3alphaRecommender: Saving model in file '/kaggle/working/best_P3alpha_tuned'
P3alphaRecommender: Saving complete
File 'TrainedModels/P3alphaRecommender/best_P3alpha_tuned.zip' updated successfully.
